In [0]:
# Databricks notebook source
# ============================================================================
# SECTION 1: NOTEBOOK CONFIGURATION AND SETUP
# ============================================================================
# This section initializes the notebook environment and sets up necessary
# configurations for the ETL pipeline

# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, upper, lower, when, count, sum, avg, 
    max, min, round, to_date, year, month, current_timestamp,
    regexp_replace, coalesce, isnan, lit, countDistinct
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    DoubleType, DateType, TimestampType
)
from pyspark.sql.window import Window
from datetime import datetime

start_notebook_datetime = datetime.now()

# Display notebook configuration
print("=" * 80)
print("ETL PIPELINE - RAW DATA TO UNITY CATALOG")
print("=" * 80)
print(f"Spark Version: {spark.version}")
print(f"Start Execution Time: {start_notebook_datetime}")

In [0]:
# ============================================================================
# SECTION 2: UNITY CATALOG CONFIGURATION
# ============================================================================
# Define the three-level namespace for Unity Catalog: catalog.schema.table
# Unity Catalog provides centralized governance for all data assets

# Define Notebook Parameters
dbutils.widgets.text("CATALOG_NAME", "workspace")
dbutils.widgets.text("SCHEMA_NAME", "portfolio_project")
dbutils.widgets.text("TABLE_NAME", "sales_summary")

# Define Unity Catalog namespace variables
CATALOG_NAME = dbutils.widgets.get("CATALOG_NAME")
SCHEMA_NAME = dbutils.widgets.get("SCHEMA_NAME")
TABLE_NAME = dbutils.widgets.get("TABLE_NAME")
FULL_TABLE_PATH = f"{CATALOG_NAME}.{SCHEMA_NAME}.{TABLE_NAME}"

# Raw data paths
RAW_CSV_PATH = "/Volumes/workspace/portfolio_projects/volume_portfolio_projects/simple_etl_project_raw_data_csv/"
RAW_PARQUET_PATH = "/Volumes/workspace/portfolio_projects/volume_portfolio_projects/simple_etl_project_raw_data_parquet/"

print(f"Unity Catalog Configuration:")
print(f"  - Catalog: {CATALOG_NAME}")
print(f"  - Schema: {SCHEMA_NAME}")
print(f"  - Table: {TABLE_NAME}")
print(f"  - Full Path: {FULL_TABLE_PATH}")

In [0]:
# ============================================================================
# SECTION 3: CREATE CATALOG AND SCHEMA
# ============================================================================
# Set up the Unity Catalog infrastructure if it doesn't exist
# This ensures proper organization and governance of data assets

# Activate the target catalog
spark.sql(f"USE CATALOG {CATALOG_NAME}")
print(f"✓ Using catalog: {CATALOG_NAME}")

# Create schema if it doesn't exist
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}
    COMMENT 'Portfolio project schema for ETL demonstration'
""")
print(f"✓ Schema created/verified: {SCHEMA_NAME}")

# Verify the schema was created successfully
schemas_df = spark.sql(f"SHOW SCHEMAS IN {CATALOG_NAME}")
display(schemas_df)

In [0]:
# ============================================================================
# SECTION 4: EXTRACT - LOAD RAW DATA
# ============================================================================
# Extract raw data from various file formats
# Databricks supports CSV, Parquet, JSON, Delta, and more

# Option 1: Load data from CSV
print("Loading data from CSV...")
df_raw = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dateFormat", "yyyy-MM-dd") \
    .option("nullValue", "NULL") \
    .option("mode", "PERMISSIVE") \
    .load(RAW_CSV_PATH)

# Option 2: Load data from Parquet (more efficient for large datasets)
# Uncomment the following lines to use Parquet instead
# print("Loading data from Parquet...")
# df_raw = spark.read \
#     .format("parquet") \
#     .load(RAW_PARQUET_PATH)

# Option 3: Load from Delta Lake (recommended for Databricks)
# df_raw = spark.read.format("delta").load("/path/to/delta/table")

print(f"✓ Raw data loaded successfully")
print(f"  - Total rows: {df_raw.count():,}")
print(f"  - Total columns: {len(df_raw.columns)}")

# Display sample of raw data
print("\nSample of raw data (first 10 rows):")
display(df_raw.limit(10))

In [0]:
# ============================================================================
# SECTION 5: DATA PROFILING AND QUALITY ASSESSMENT
# ============================================================================
# Before cleaning, it's crucial to understand data quality issues
# This section provides comprehensive data profiling

print("=" * 80)
print("DATA QUALITY ASSESSMENT")
print("=" * 80)

# Check for null values in each column
print("\n1. NULL VALUE ANALYSIS:")
null_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_raw.columns
])
display(null_counts)

# Check for duplicate records
duplicate_count = df_raw.count() - df_raw.dropDuplicates().count()
print(f"\n2. DUPLICATE RECORDS: {duplicate_count:,}")

# Basic statistical summary
print("\n3. STATISTICAL SUMMARY:")
display(df_raw.describe())

# Check data types
print("\n4. CURRENT DATA TYPES:")
df_raw.printSchema()

In [0]:
# ============================================================================
# SECTION 6: TRANSFORM - DATA CLEANING
# ============================================================================
# Apply comprehensive data cleaning operations
# Each step is explained and can be customized based on your data

print("Starting data cleaning process...")

# Step 1: Remove exact duplicate rows
df_cleaned = df_raw.dropDuplicates()
print(f"✓ Step 1: Removed {df_raw.count() - df_cleaned.count()} duplicate rows")

# Step 2: Handle missing values with business logic
# Different strategies for different columns
df_cleaned = df_cleaned \
    .na.fill({
        "quantity": 0,          # Fill missing quantities with 0
        "unit_price": 0.0,      # Fill missing prices with 0
        "discount": 0.0,        # Assume no discount if missing
        "category": "Unknown",  # Default category
        "region": "Unspecified" # Default region
    })
print("✓ Step 2: Filled missing values with appropriate defaults")

# Step 3: Drop rows where critical fields are null
# Transaction ID and Date are mandatory for our analysis
df_cleaned = df_cleaned \
    .filter(col("transaction_id").isNotNull()) \
    .filter(col("transaction_date").isNotNull()) \
    .filter(col("customer_id").isNotNull())
print("✓ Step 3: Removed rows with null critical fields")

# Step 4: Standardize text fields (trim whitespace, consistent casing)
df_cleaned = df_cleaned \
    .withColumn("customer_name", trim(upper(col("customer_name")))) \
    .withColumn("product_name", trim(upper(col("product_name")))) \
    .withColumn("category", trim(upper(col("category")))) \
    .withColumn("region", trim(upper(col("region")))) \
    .withColumn("sales_person", trim(upper(col("sales_person"))))
print("✓ Step 4: Standardized text fields (uppercase and trimmed)")

# Step 5: Convert date strings to proper date type
df_cleaned = df_cleaned \
    .withColumn("transaction_date", to_date(col("transaction_date"), "yyyy-MM-dd"))
print("✓ Step 5: Converted transaction_date to proper DateType")

# Step 6: Add derived columns for better analysis
df_cleaned = df_cleaned \
    .withColumn("year", year(col("transaction_date"))) \
    .withColumn("month", month(col("transaction_date"))) \
    .withColumn("total_amount", 
                round((col("quantity") * col("unit_price")) * (1 - col("discount")), 2))
print("✓ Step 6: Added derived columns (year, month, total_amount)")

# Step 7: Data validation - remove invalid records
# Ensure quantity and price are positive
df_cleaned = df_cleaned \
    .filter(col("quantity") >= 0) \
    .filter(col("unit_price") >= 0) \
    .filter(col("discount") >= 0) \
    .filter(col("discount") <= 1)  # Discount should be between 0 and 1
print("✓ Step 7: Filtered out invalid records (negative values, invalid discounts)")

# Step 8: Remove outliers (optional - adjust thresholds based on your data)
# For example, remove transactions with unrealistic quantities
df_cleaned = df_cleaned.filter(col("quantity") <= 1000)
print("✓ Step 8: Removed outliers (quantity > 1000)")

print(f"\nCleaning complete:")
print(f"  - Original rows: {df_raw.count():,}")
print(f"  - Cleaned rows: {df_cleaned.count():,}")
print(f"  - Rows removed: {df_raw.count() - df_cleaned.count():,}")
print(f"  - Data retention rate: {(df_cleaned.count() / df_raw.count() * 100):.2f}%")

# Display sample of cleaned data
print("\nSample of cleaned data:")
display(df_cleaned.limit(10))

In [0]:
# ============================================================================
# SECTION 7: TRANSFORM - DATA AGGREGATION
# ============================================================================
# Create aggregated views of the data for analytical purposes
# This demonstrates various aggregation techniques in PySpark

print("Creating aggregated dataset...")

# Aggregation 1: Monthly sales summary by category and region
df_aggregated = df_cleaned.groupBy(
    "year",
    "month",
    "category",
    "region"
).agg(
    count("transaction_id").alias("total_transactions"),
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_revenue"),
    avg("total_amount").alias("avg_transaction_value"),
    max("total_amount").alias("max_transaction_value"),
    min("total_amount").alias("min_transaction_value"),
    countDistinct("customer_id").alias("unique_customers"),
    countDistinct("product_id").alias("unique_products")
).orderBy("year", "month", "category", "region")

# Round decimal values for better readability
df_aggregated = df_aggregated \
    .withColumn("total_revenue", round(col("total_revenue"), 2)) \
    .withColumn("avg_transaction_value", round(col("avg_transaction_value"), 2)) \
    .withColumn("max_transaction_value", round(col("max_transaction_value"), 2)) \
    .withColumn("min_transaction_value", round(col("min_transaction_value"), 2))

# Add metadata columns for tracking
df_aggregated = df_aggregated \
    .withColumn("created_at", current_timestamp()) \
    .withColumn("data_source", lit("sales_etl_pipeline"))

print(f"✓ Aggregation complete")
print(f"  - Aggregated rows: {df_aggregated.count():,}")
print(f"  - Aggregation dimensions: year, month, category, region")
print(f"  - Metrics calculated: 8 business metrics")

# Display aggregated data
print("\nSample of aggregated data:")
display(df_aggregated.limit(20))

# Show aggregation summary
print("\nAggregation summary by category:")
display(
    df_aggregated.groupBy("category")
    .agg(
        sum("total_revenue").alias("category_revenue"),
        sum("total_transactions").alias("category_transactions")
    )
    .orderBy(col("category_revenue").desc())
)

In [0]:
# ============================================================================
# SECTION 8: LOAD - WRITE TO UNITY CATALOG WITH SMART PARTITIONING
# ============================================================================
# Save the cleaned and aggregated data to Unity Catalog
# Using Delta Lake format with automatic partition detection and selective overwrite

from datetime import datetime

print("=" * 80)
print("WRITING TO UNITY CATALOG - PARTITIONED TABLE")
print("=" * 80)
print(f"Target: {FULL_TABLE_PATH}")

# ============================================================================
# STEP 1: ANALYZE PARTITIONS IN INPUT DATA
# ============================================================================

# Automatically detect which partitions are present in the incoming data
partitions_df = df_aggregated.select("year", "month").distinct().orderBy("year", "month")
partitions_list = partitions_df.collect()

print(f"\n📊 Partition Analysis:")
print(f"  - Total partitions in input data: {len(partitions_list)}")
print(f"  - Partition columns: year, month")
print(f"\n  Partitions to be overwritten:")

# Build the replaceWhere clause dynamically
partition_conditions = []
for partition in partitions_list:
    year_val = partition.year
    month_val = partition.month
    print(f"    • year={year_val}, month={month_val}")
    partition_conditions.append(f"(year = {year_val} AND month = {month_val})")

# Combine all partition conditions with OR
replace_where_clause = " OR ".join(partition_conditions)
print(f"\n  Generated replaceWhere clause:")
print(f"    {replace_where_clause[:150]}{'...' if len(replace_where_clause) > 150 else ''}")

# ============================================================================
# STEP 2: COLLECT PRE-WRITE STATISTICS (if table exists)
# ============================================================================

# Check if table already exists to collect statistics
table_exists = spark.catalog.tableExists(FULL_TABLE_PATH)

if table_exists:
    print(f"\n📈 Pre-Write Statistics:")
    
    # Total row count before
    pre_total_count = spark.sql(f"SELECT COUNT(*) as cnt FROM {FULL_TABLE_PATH}").collect()[0].cnt
    print(f"  - Total rows before: {pre_total_count:,}")
    
    # Count rows in partitions that will be overwritten
    pre_partition_count = spark.sql(f"""
        SELECT COUNT(*) as cnt 
        FROM {FULL_TABLE_PATH} 
        WHERE {replace_where_clause}
    """).collect()[0].cnt
    print(f"  - Rows in target partitions before: {pre_partition_count:,}")
    
    # Partition distribution before
    print(f"\n  Current partition distribution:")
    pre_partition_dist = spark.sql(f"""
        SELECT year, month, COUNT(*) as row_count
        FROM {FULL_TABLE_PATH}
        GROUP BY year, month
        ORDER BY year DESC, month DESC
    """)
    display(pre_partition_dist)
    
    # Table size estimation
    table_details = spark.sql(f"DESCRIBE DETAIL {FULL_TABLE_PATH}").collect()[0]
    pre_size_bytes = table_details.sizeInBytes if hasattr(table_details, 'sizeInBytes') else 0
    pre_num_files = table_details.numFiles if hasattr(table_details, 'numFiles') else 0
    print(f"\n  Table storage metrics:")
    print(f"    - Size: {pre_size_bytes / (1024**2):.2f} MB")
    print(f"    - Number of files: {pre_num_files}")
else:
    print(f"\n📊 Table does not exist yet - will be created")
    pre_total_count = 0
    pre_partition_count = 0
    pre_size_bytes = 0
    pre_num_files = 0

# ============================================================================
# STEP 3: WRITE DATA WITH SELECTIVE PARTITION OVERWRITE
# ============================================================================

write_start_time = datetime.now()
print(f"\n⏳ Starting write operation at {write_start_time.strftime('%H:%M:%S')}")

# Count rows being written
rows_to_write = df_aggregated.count()
print(f"  - Rows to write: {rows_to_write:,}")

# Write with replaceWhere for selective partition overwrite
df_aggregated.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .option("replaceWhere", replace_where_clause) \
    .option("overwriteSchema", "false") \
    .option("delta.columnMapping.mode", "name") \
    .saveAsTable(FULL_TABLE_PATH)

write_end_time = datetime.now()
write_duration = (write_end_time - write_start_time).total_seconds()

print(f"✓ Data successfully written to Unity Catalog")
print(f"  - Write duration: {write_duration:.2f} seconds")
print(f"  - Throughput: {rows_to_write/write_duration:,.0f} rows/second")

# ============================================================================
# STEP 4: POST-WRITE VALIDATION AND STATISTICS
# ============================================================================

print(f"\n📊 Post-Write Statistics:")

# Total row count after
post_total_count = spark.sql(f"SELECT COUNT(*) as cnt FROM {FULL_TABLE_PATH}").collect()[0].cnt
print(f"  - Total rows after: {post_total_count:,}")

# Count rows in affected partitions
post_partition_count = spark.sql(f"""
    SELECT COUNT(*) as cnt 
    FROM {FULL_TABLE_PATH} 
    WHERE {replace_where_clause}
""").collect()[0].cnt
print(f"  - Rows in target partitions after: {post_partition_count:,}")

# Calculate changes
if table_exists:
    rows_added = post_total_count - pre_total_count
    rows_replaced = pre_partition_count
    print(f"\n  Changes:")
    print(f"    • Rows in affected partitions replaced: {rows_replaced:,}")
    print(f"    • Net change in total rows: {rows_added:+,}")
    print(f"    • Rows in unaffected partitions: {post_total_count - post_partition_count:,} (unchanged)")

# Partition distribution after
print(f"\n  Updated partition distribution:")
post_partition_dist = spark.sql(f"""
    SELECT 
        year, 
        month, 
        COUNT(*) as row_count,
        ROUND(SUM(total_revenue), 2) as partition_revenue,
        COUNT(DISTINCT category) as distinct_categories,
        COUNT(DISTINCT region) as distinct_regions
    FROM {FULL_TABLE_PATH}
    GROUP BY year, month
    ORDER BY year DESC, month DESC
""")
display(post_partition_dist)

# ============================================================================
# STEP 5: TABLE METADATA AND PROPERTIES
# ============================================================================

print(f"\n⚙️  Configuring table properties...")

# Add table comment with detailed metadata
spark.sql(f"""
    COMMENT ON TABLE {FULL_TABLE_PATH} IS 
    'Aggregated sales data by month, category, and region.
    
    Partitioning: year, month
    Granularity: Monthly aggregations
    Source: ETL pipeline from raw sales data
    Last Updated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    Rows: {post_total_count:,}
    Partitions: {len(partitions_list)}'
""")

# Enable auto-optimization features
spark.sql(f"""
    ALTER TABLE {FULL_TABLE_PATH} SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true',
        'delta.logRetentionDuration' = '30 days',
        'delta.deletedFileRetentionDuration' = '7 days'
    )
""")
print("✓ Auto-optimize and retention policies configured")

# ============================================================================
# STEP 6: OPTIMIZE TABLE FOR QUERY PERFORMANCE
# ============================================================================

print(f"\n🚀 Optimizing table for query performance...")

optimize_start_time = datetime.now()

# Optimize with Z-ORDER on frequently queried columns
spark.sql(f"""
    OPTIMIZE {FULL_TABLE_PATH} 
    ZORDER BY (category, region)
""")

optimize_end_time = datetime.now()
optimize_duration = (optimize_end_time - optimize_start_time).total_seconds()

print(f"✓ OPTIMIZE completed in {optimize_duration:.2f} seconds")
print(f"  - Z-ORDER applied on: category, region")
print(f"  - Benefit: Faster filtering on these columns")

# ============================================================================
# STEP 7: COMPUTE TABLE STATISTICS
# ============================================================================

print(f"\n📊 Computing statistics for query optimizer...")

stats_start_time = datetime.now()

# Compute comprehensive statistics for all columns
spark.sql(f"ANALYZE TABLE {FULL_TABLE_PATH} COMPUTE STATISTICS FOR ALL COLUMNS")

stats_end_time = datetime.now()
stats_duration = (stats_end_time - stats_start_time).total_seconds()

print(f"✓ Statistics computed in {stats_duration:.2f} seconds")
print(f"  - Benefit: Better query execution plans")

# ============================================================================
# STEP 8: DETAILED TABLE INSIGHTS
# ============================================================================

print("\n" + "=" * 80)
print("TABLE INSIGHTS AND PERFORMANCE METRICS")
print("=" * 80)

# Get detailed table information
table_details = spark.sql(f"DESCRIBE DETAIL {FULL_TABLE_PATH}").collect()[0]

post_size_bytes = table_details.sizeInBytes if hasattr(table_details, 'sizeInBytes') else 0
post_num_files = table_details.numFiles if hasattr(table_details, 'numFiles') else 0

print(f"\n📦 Storage Metrics:")
print(f"  - Table size: {post_size_bytes / (1024**2):.2f} MB")
print(f"  - Number of files: {post_num_files}")
print(f"  - Average file size: {post_size_bytes / post_num_files / (1024**2):.2f} MB" if post_num_files > 0 else "  - Average file size: N/A")
print(f"  - Format: Delta Lake (Parquet + transaction log)")

if table_exists:
    size_change = post_size_bytes - pre_size_bytes
    file_change = post_num_files - pre_num_files
    print(f"\n  Storage changes:")
    print(f"    • Size change: {size_change / (1024**2):+.2f} MB")
    print(f"    • File count change: {file_change:+d}")

# Show partition file distribution
print(f"\n📁 Partition File Distribution:")
partition_details = spark.sql(f"""
    SELECT year, month, COUNT(*) as record_count
    FROM {FULL_TABLE_PATH}
    GROUP BY year, month
    ORDER BY year DESC, month DESC
""")
display(partition_details)

# Show physical partition paths
print(f"\n🗂️  Physical Partitions:")
physical_partitions = spark.sql(f"SHOW PARTITIONS {FULL_TABLE_PATH}").limit(10)
display(physical_partitions)

In [0]:
# ============================================================================
# STEP 9: QUERY PERFORMANCE TESTING
# ============================================================================

print(f"\n⚡ Query Performance Testing:")

# Test 1: Full table scan
test1_start = datetime.now()
full_scan_count = spark.sql(f"SELECT COUNT(*) FROM {FULL_TABLE_PATH}").collect()[0][0]
test1_duration = (datetime.now() - test1_start).total_seconds()
print(f"  Test 1 - Full table scan: {test1_duration:.3f}s ({full_scan_count:,} rows)")

# Test 2: Single partition query (partition pruning)
if len(partitions_list) > 0:
    test_year = partitions_list[0].year
    test_month = partitions_list[0].month
    test2_start = datetime.now()
    partition_count = spark.sql(f"""
        SELECT COUNT(*) FROM {FULL_TABLE_PATH} 
        WHERE year = {test_year} AND month = {test_month}
    """).collect()[0][0]
    test2_duration = (datetime.now() - test2_start).total_seconds()
    print(f"  Test 2 - Single partition (year={test_year}, month={test_month}): {test2_duration:.3f}s ({partition_count:,} rows)")
    
    if test2_duration > 0:
        print(f"    → Speedup from partition pruning: {test1_duration/test2_duration:.1f}x faster")

# Test 3: Z-ORDER column filter
test3_start = datetime.now()
category_count = spark.sql(f"""
    SELECT COUNT(*) FROM {FULL_TABLE_PATH} 
    WHERE category = 'ELECTRONICS'
""").collect()[0][0]
test3_duration = (datetime.now() - test3_start).total_seconds()
print(f"  Test 3 - Category filter (Z-ORDERed): {test3_duration:.3f}s ({category_count:,} rows)")

In [0]:
# ============================================================================
# STEP 10: TABLE HISTORY AND VERSION CONTROL
# ============================================================================

print(f"\n📜 Table History (Delta Time Travel):")

# Show last 5 operations
history_df = spark.sql(f"DESCRIBE HISTORY {FULL_TABLE_PATH}").limit(5)
display(history_df.select(
    "version", 
    "timestamp", 
    "operation", 
    "operationMetrics",
    "userName"
))

# Get current version
current_version = spark.sql(f"DESCRIBE HISTORY {FULL_TABLE_PATH}").first().version
print(f"\n  Current table version: {current_version}")
print(f"  You can time travel to any previous version using:")
print(f"    SELECT * FROM {FULL_TABLE_PATH} VERSION AS OF <version>")

In [0]:
# ============================================================================
# STEP 11: DATA QUALITY VALIDATION
# ============================================================================

print(f"\n✅ Data Quality Checks:")

# Check 1: No null values in partition columns
null_partitions = spark.sql(f"""
    SELECT COUNT(*) as null_count
    FROM {FULL_TABLE_PATH}
    WHERE year IS NULL OR month IS NULL
""").collect()[0].null_count

print(f"  ✓ Null partition keys: {null_partitions} (should be 0)")

# Check 2: Validate aggregation logic
print(f"\n  Business metrics validation:")
validation_df = spark.sql(f"""
    SELECT 
        COUNT(DISTINCT year) as distinct_years,
        COUNT(DISTINCT month) as distinct_months,
        COUNT(DISTINCT category) as distinct_categories,
        COUNT(DISTINCT region) as distinct_regions,
        MIN(total_transactions) as min_transactions,
        MAX(total_transactions) as max_transactions,
        ROUND(SUM(total_revenue), 2) as grand_total_revenue
    FROM {FULL_TABLE_PATH}
""")
display(validation_df)

# Check 3: Top performers
print(f"\n  Top 5 performing segments:")
top_performers = spark.sql(f"""
    SELECT 
        year, month, category, region,
        ROUND(total_revenue, 2) as revenue,
        total_transactions as transactions
    FROM {FULL_TABLE_PATH}
    ORDER BY total_revenue DESC
    LIMIT 5
""")
display(top_performers)

In [0]:
# Databricks notebook source
# ============================================================================
# SECTION 12: ADD TAGS, COMMENTS AND METADATA TO TABLE AND COLUMNS
# ============================================================================
# Enhance data discoverability and governance by adding comprehensive metadata
# Tags and comments help data consumers understand the data and its usage

from datetime import datetime

print("=" * 80)
print("ADDING METADATA: TAGS, COMMENTS AND DESCRIPTIONS")
print("=" * 80)
print(f"Target: {FULL_TABLE_PATH}")

# ============================================================================
# STEP 1: ADD COLUMN COMMENTS (DESCRIPTIONS)
# ============================================================================

print("\n📝 Adding column descriptions...")

# Define column comments for data documentation
# These descriptions appear in Catalog Explorer and DESCRIBE commands
column_comments = {
    "year": "Transaction year - Used for partitioning",
    "month": "Transaction month (1-12) - Used for partitioning",
    "category": "Product category - Z-ORDERed for query optimization",
    "region": "Geographic region where the sale occurred - Z-ORDERed for query optimization",
    "total_transactions": "Total number of individual transactions in this segment",
    "total_quantity_sold": "Sum of all quantities sold across all transactions",
    "total_revenue": "Total revenue generated (quantity * unit_price * (1-discount))",
    "avg_transaction_value": "Average revenue per transaction in this segment",
    "max_transaction_value": "Maximum single transaction value in this segment",
    "min_transaction_value": "Minimum single transaction value in this segment",
    "unique_customers": "Count of distinct customers who made purchases",
    "unique_products": "Count of distinct products sold in this segment",
    "created_at": "Timestamp when this record was created in the table",
    "data_source": "Source system or pipeline that created this record"
}

# Apply comments to all columns at once (supports multiple columns in one command)
alter_column_comments = []
for column, comment in column_comments.items():
    # Escape single quotes in comments to prevent SQL injection
    escaped_comment = comment.replace("'", "''")
    alter_column_comments.append(f"{column} COMMENT '{escaped_comment}'")

# Execute ALTER TABLE with all column comments
alter_statement = f"""
    ALTER TABLE {FULL_TABLE_PATH} ALTER COLUMN
    {',\n    '.join(alter_column_comments)}
"""

spark.sql(alter_statement)
print(f"✓ Added descriptions to {len(column_comments)} columns")

# Verify comments were added
print("\nColumn descriptions preview:")
describe_df = spark.sql(f"DESCRIBE TABLE {FULL_TABLE_PATH}")
display(describe_df.select("col_name", "data_type", "comment").limit(10))

# ============================================================================
# STEP 2: ADD COLUMN TAGS
# ============================================================================

print("\n🏷️  Adding column tags for governance...")

# Define tags for each column
# Tags help with data classification, security, and compliance
# Common tag categories: PII, Sensitivity, Data Quality, Business Domain

column_tags = {
    "year": {"category": "partition_key", "data_classification": "public"},
    "month": {"category": "partition_key", "data_classification": "public"},
    "category": {"category": "business_dimension", "data_classification": "public", "indexed": "z_order"},
    "region": {"category": "business_dimension", "data_classification": "public", "indexed": "z_order"},
    "total_transactions": {"category": "kpi", "metric_type": "count", "data_classification": "internal"},
    "total_quantity_sold": {"category": "kpi", "metric_type": "sum", "data_classification": "internal"},
    "total_revenue": {"category": "kpi", "metric_type": "currency", "data_classification": "confidential", "pii": "false"},
    "avg_transaction_value": {"category": "kpi", "metric_type": "average", "data_classification": "internal"},
    "max_transaction_value": {"category": "kpi", "metric_type": "max", "data_classification": "internal"},
    "min_transaction_value": {"category": "kpi", "metric_type": "min", "data_classification": "internal"},
    "unique_customers": {"category": "kpi", "metric_type": "distinct_count", "data_classification": "internal"},
    "unique_products": {"category": "kpi", "metric_type": "distinct_count", "data_classification": "internal"},
    "created_at": {"category": "metadata", "data_classification": "public"},
    "data_source": {"category": "metadata", "data_classification": "public"}
}

# Apply tags to each column (must be done separately per column)
tag_count = 0
for column, tags in column_tags.items():
    # Convert tags dict to proper format: ('key' = 'value', 'key2' = 'value2')
    tag_pairs = [f"'{k}' = '{v}'" for k, v in tags.items()]
    tags_string = ", ".join(tag_pairs)
    
    # Apply tags to column
    spark.sql(f"""
        ALTER TABLE {FULL_TABLE_PATH} 
        ALTER COLUMN {column} 
        SET TAGS ({tags_string})
    """)
    tag_count += len(tags)
    print(f"  ✓ Tagged column '{column}' with {len(tags)} tags")

print(f"\n✓ Applied {tag_count} total tags across {len(column_tags)} columns")

# ============================================================================
# STEP 3: ADD TABLE-LEVEL TAGS
# ============================================================================

print("\n🏷️  Adding table-level tags...")

# Define table-level tags for governance and classification
table_tags = {
    "data_domain": "sales_analytics",
    "data_owner": "data_engineering_team",
    "refresh_frequency": "daily",
    "data_classification": "internal",
    "contains_pii": "false",
    "quality_tier": "gold",
    "business_criticality": "high",
    "retention_policy": "3_years",
    "compliance": "gdpr_compliant",
    "sla": "24_hours",
    "table_type": "aggregated",
    "partition_strategy": "year_month",
    "optimization": "z_order_enabled"
}

# Convert to proper format
table_tag_pairs = [f"'{k}' = '{v}'" for k, v in table_tags.items()]
table_tags_string = ", ".join(table_tag_pairs)

# Apply tags to table
spark.sql(f"""
    ALTER TABLE {FULL_TABLE_PATH} 
    SET TAGS ({table_tags_string})
""")

print(f"✓ Applied {len(table_tags)} tags to table")
print("\nTable tags:")
for key, value in table_tags.items():
    print(f"  • {key}: {value}")

# ============================================================================
# STEP 4: UPDATE TABLE DESCRIPTION (EXTENDED COMMENT)
# ============================================================================

print("\n📄 Updating table description...")

# Create comprehensive table description with Markdown support
table_description = f"""
# Sales Summary Analytics Table

## Overview
Aggregated sales data providing monthly insights by category and region. 
This table serves as the primary source for sales performance dashboards and reporting.

## Data Characteristics
- **Granularity**: Monthly aggregations
- **Dimensions**: Year, Month, Category, Region
- **Metrics**: Revenue, Transactions, Quantities, Customer counts
- **Partitioning**: Year and Month for optimal time-based queries
- **Optimization**: Z-ORDER on Category and Region

## Business Use Cases
1. Monthly sales performance tracking
2. Category and regional comparison analysis
3. Customer behavior insights
4. Revenue forecasting and planning

## Technical Details
- **Source**: Raw transactional sales data (CSV/Parquet)
- **ETL Pipeline**: Automated data cleaning and aggregation
- **Update Frequency**: Daily incremental loads
- **Data Quality**: Validated and cleaned
- **Time Travel**: Enabled (30 days retention)

## Data Lineage
Raw Sales Data → Data Cleaning → Aggregation → Unity Catalog Table

## Owner & Contact
- **Team**: Data Engineering
- **Contact**: data-eng@company.com
- **Last Updated**: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
- **Version**: 1.0

## Important Notes
- Partitioned by year/month - always include these in WHERE clauses for best performance
- Z-ORDER optimized for category and region filters
- Revenue excludes returns and cancellations
- Historical data available via Delta Lake time travel
"""

# Apply comprehensive comment to table (supports Markdown)
escaped_description = table_description.replace("'", "''")
spark.sql(f"""
    COMMENT ON TABLE {FULL_TABLE_PATH} IS '{escaped_description}'
""")

print("✓ Table description updated with comprehensive documentation")

# ============================================================================
# STEP 5: VERIFY ALL METADATA
# ============================================================================

print("\n" + "=" * 80)
print("METADATA VERIFICATION")
print("=" * 80)

# Show table details with comments
print("\n1. Table Information:")
table_info = spark.sql(f"DESCRIBE EXTENDED {FULL_TABLE_PATH}")
display(table_info)

# Show column details with comments and data types
print("\n2. Column Details with Descriptions:")
column_details = spark.sql(f"DESCRIBE TABLE {FULL_TABLE_PATH}")
display(column_details)

# Query table tags from INFORMATION_SCHEMA
print("\n3. Table Tags:")
table_tags_query = spark.sql(f"""
    SELECT 
        tag_name,
        tag_value
    FROM {CATALOG_NAME}.INFORMATION_SCHEMA.TABLE_TAGS
    WHERE catalog_name = '{CATALOG_NAME}'
    AND schema_name = '{SCHEMA_NAME}'
    AND table_name = '{TABLE_NAME}'
    ORDER BY tag_name
""")
display(table_tags_query)

# Query column tags from INFORMATION_SCHEMA
print("\n4. Column Tags:")
column_tags_query = spark.sql(f"""
    SELECT 
        column_name,
        tag_name,
        tag_value
    FROM {CATALOG_NAME}.INFORMATION_SCHEMA.COLUMN_TAGS
    WHERE catalog_name = '{CATALOG_NAME}'
    AND schema_name = '{SCHEMA_NAME}'
    AND table_name = '{TABLE_NAME}'
    ORDER BY column_name, tag_name
""")
display(column_tags_query)

# ============================================================================
# STEP 6: CREATE METADATA SUMMARY REPORT
# ============================================================================

print("\n📊 Metadata Summary Report:")

# Count total metadata elements
total_column_comments = len(column_comments)
total_column_tags = tag_count
total_table_tags = len(table_tags)

print(f"\n  Documentation Metrics:")
print(f"    • Columns documented: {total_column_comments}/{len(df_aggregated.columns)}")
print(f"    • Column tags applied: {total_column_tags}")
print(f"    • Table tags applied: {total_table_tags}")
print(f"    • Table description: ✓ Comprehensive Markdown documentation")

# Create a summary DataFrame
metadata_summary = spark.sql(f"""
    SELECT 
        '{FULL_TABLE_PATH}' as table_path,
        COUNT(DISTINCT c.column_name) as documented_columns,
        COUNT(DISTINCT ct.column_name) as tagged_columns,
        COUNT(DISTINCT ct.tag_name) as total_column_tags,
        '{len(table_tags)}' as table_level_tags
    FROM {CATALOG_NAME}.INFORMATION_SCHEMA.COLUMNS c
    LEFT JOIN {CATALOG_NAME}.INFORMATION_SCHEMA.COLUMN_TAGS ct
        ON c.table_catalog = ct.catalog_name
        AND c.table_schema = ct.schema_name
        AND c.table_name = ct.table_name
        AND c.column_name = ct.column_name
    WHERE c.table_catalog = '{CATALOG_NAME}'
    AND c.table_schema = '{SCHEMA_NAME}'
    AND c.table_name = '{TABLE_NAME}'
""")

display(metadata_summary)

# ============================================================================
# STEP 7: METADATA BEST PRACTICES VALIDATION
# ============================================================================

print("\n✅ Metadata Best Practices Check:")

# Check 1: All columns have comments
columns_without_comments = spark.sql(f"""
    SELECT column_name
    FROM {CATALOG_NAME}.INFORMATION_SCHEMA.COLUMNS
    WHERE table_catalog = '{CATALOG_NAME}'
    AND table_schema = '{SCHEMA_NAME}'
    AND table_name = '{TABLE_NAME}'
    AND (comment IS NULL OR comment = '')
""").count()

if columns_without_comments == 0:
    print("  ✓ All columns have descriptions")
else:
    print(f"  ⚠️  {columns_without_comments} columns missing descriptions")

# Check 2: Critical columns have tags
critical_columns = ['total_revenue', 'unique_customers']
tagged_critical = spark.sql(f"""
    SELECT DISTINCT column_name
    FROM {CATALOG_NAME}.INFORMATION_SCHEMA.COLUMN_TAGS
    WHERE catalog_name = '{CATALOG_NAME}'
    AND schema_name = '{SCHEMA_NAME}'
    AND table_name = '{TABLE_NAME}'
    AND column_name IN ('total_revenue', 'unique_customers')
""").count()

if tagged_critical == len(critical_columns):
    print(f"  ✓ All critical columns are tagged")
else:
    print(f"  ⚠️  Some critical columns need tags")

# Check 3: Table has governance tags
governance_tags = ['data_classification', 'data_owner', 'contains_pii']
has_governance = spark.sql(f"""
    SELECT COUNT(DISTINCT tag_name) as gov_tag_count
    FROM {CATALOG_NAME}.INFORMATION_SCHEMA.TABLE_TAGS
    WHERE catalog_name = '{CATALOG_NAME}'
    AND schema_name = '{SCHEMA_NAME}'
    AND table_name = '{TABLE_NAME}'
    AND tag_name IN ('data_classification', 'data_owner', 'contains_pii')
""").collect()[0].gov_tag_count

if has_governance >= 3:
    print(f"  ✓ Table has all required governance tags")
else:
    print(f"  ⚠️  Table missing some governance tags")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("✅ METADATA ENRICHMENT COMPLETED")
print("=" * 80)

print(f"\n📋 Summary:")
print(f"  • Table: {FULL_TABLE_PATH}")
print(f"  • Column descriptions: {total_column_comments} added")
print(f"  • Column tags: {total_column_tags} applied")
print(f"  • Table tags: {total_table_tags} applied")
print(f"  • Table documentation: ✓ Comprehensive Markdown description")

print(f"\n🎯 Benefits:")
print(f"  ✓ Improved data discoverability in Catalog Explorer")
print(f"  ✓ Enhanced data governance and compliance tracking")
print(f"  ✓ Better data lineage and ownership transparency")
print(f"  ✓ Easier onboarding for new data consumers")
print(f"  ✓ Automated data classification for security policies")

print(f"\n📖 View Metadata:")
print(f"  • Catalog Explorer: Browse to {FULL_TABLE_PATH}")
print(f"  • SQL: DESCRIBE EXTENDED {FULL_TABLE_PATH}")
print(f"  • SQL: SELECT * FROM {CATALOG_NAME}.INFORMATION_SCHEMA.TABLE_TAGS")
print(f"  • SQL: SELECT * FROM {CATALOG_NAME}.INFORMATION_SCHEMA.COLUMN_TAGS")

print("\n" + "=" * 80)


In [0]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("=" * 80)
print("ETL PIPELINE COMPLETED SUCCESSFULLY")
print("=" * 80)
print(f"\n📊 Summary:")
print(f"  • Source: Raw CSV/Parquet files")
print(f"  • Destination: {FULL_TABLE_PATH}")
print(f"  • Records processed: {df_raw.count():,}")
print(f"  • Records loaded: {table_count:,}")
print(f"  • Data quality: ✓ Cleaned and validated")
print(f"  • Format: Delta Lake")
print(f"  • Governance: Unity Catalog")
print(f"  • Start Execution DateTime: {start_notebook_datetime}")
print(f"  • End Execution DateTime: {end_notebook_datetime}")
print(f"  • Execution Time: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
print(f"\n✅ Data is now available for analytics and reporting")
print(f"\nNext steps:")
print(f"  1. Query the table: SELECT * FROM {FULL_TABLE_PATH}")
print(f"  2. Create dashboards using Databricks SQL")
print(f"  3. Set up automated jobs for regular updates")
print(f"  4. Configure alerts for data quality monitoring")